In [1]:
import pandas as pd

In [2]:
# baseline_model/test_results.txt
# co-models/ext_1000_isw_model/test_results.txt = 1521
# co-models/ext_2290_isw_model/test_results.txt = 2290

# co-models/
# ext_1000_isw_model		ext_4847_isw_model
# ext_1000_top20_isw_model	ext_504_isw_model
# ext_2290_isw_model		ext_isw_model
# ext_3132_isw_model

## Comapre the test result of baseline model and co-train model
#### This will give us a insight about which NER tags are better or worse...

In [3]:
def load_eval_result(filename):
    with open(filename, encoding='utf-8') as f:
        lines = (line.strip() for line in f)
        eval_ls = []
        for line in lines:
            splits = line.split("    ")
            splits = list(filter(None, splits))
            eval_ls.append(splits)
    eval_ls = eval_ls[2:-3]+[eval_ls[-1]]

    float_list = []
    for ls in eval_ls:
        tmp_ls = []
        for ele in ls:
            try:
                ele = float(ele)
                tmp_ls.append(ele)
            except:
                tmp_ls.append(ele)
        float_list.append(tmp_ls)
    float_list = sorted(float_list, key = lambda x: x[0], reverse=False)
    return float_list

def get_better_worse_tags(baseline_ls, cotrain_ls):
    better_ = []
    worse_ = []
    for (b_tag, b_p, b_r, b_f, b_s), (tag, p, r, f, s) in zip(baseline_ls, cotrain_ls):
        assert b_tag == tag
        # better f1 score results
        if f>b_f:
            better_.append([b_tag, b_f, f])
        elif f<b_f:
            worse_.append([b_tag, b_f, f])
    return better_, worse_

In [4]:
# DF format for baseline model result
# df = pd.DataFrame(baseline_ls[0:],columns=['Baseline model','precision', 'recall','f1-score','support'])
# df

## Compare with tag distribution 
### This will give us an insight about the distribution of the new labels added to the training set, and in comparison the distribution of labels in the training set before / after adding the new labels)

In [5]:
from collections import Counter
import joblib

def get_stat(labels):
    flat_list = [item for sublist in labels for item in sublist if item != 'O']
    strList = list(map( lambda x: x.replace( 'B-', ''), flat_list))
    strList = list(map( lambda x: x.replace( 'I-', ''), strList))

    tag_list = list(Counter(strList).keys())
    num_tags = list(Counter(strList).values())
    dict_tags = dict(zip(tag_list, num_tags))

    final_tag_dict = sorted(dict_tags.items(), key=lambda x: x[1], reverse=True)
    return final_tag_dict

def get_compare_df(ori_df, train_tags, ext_data_dir, ext_, better_, worse_):
    com_tags, better_tags, worse_tags = [], [], []
    for (tag, _) in train_tags:
        better_ls = [tag for [tag, *_] in better_]
        worse_ls = [tag for [tag, *_] in worse_]
        if tag in list(ext_.keys()):
            com_tags.append(ext_[tag])
        else:
            com_tags.append(0)
        
        if tag in better_ls:
            better_tags.append(1)
        else:
            better_tags.append(0)
            
        if tag in worse_ls:
            worse_tags.append(1)
        else:
            worse_tags.append(0)
    
    ori_df[ext_data_dir]=com_tags
    ori_df['Better']=better_tags
    ori_df['Worse']=worse_tags
    return ori_df    


In [6]:
# Load better and wrose result 

# Load test result of baseline and cotrain model
baseline_ls = load_eval_result("baseline_model/test_results.txt")

# _1521_ls = load_eval_result("co-models/ext_1000_isw_model/test_results.txt")
_2290_ls = load_eval_result("co-models/ext_2290_isw_model/test_results.txt")

# Shows the better and worse tags [[PER, 0.8, 0.9]]
better_, worse_ = get_better_worse_tags(baseline_ls, _2290_ls)



# Load origin train data and new adding ext data from co-training
# Ori train data
train_sents = joblib.load("data/train-isw-sentences.pkl")
train_labels = joblib.load("data/train-isw-labels.pkl")

# Extended train data, which generated from co-trainig method.
# You should change the ext_data_dir to pick new adding train data.

ext_data_dir = "ext_data_1000_u_300"

# ext = joblib.load("ext_data/ext_data_1000/1521_ext_L_A_labels.pkl")
ext = joblib.load("ext_data/{}/2290_ext_L_A_labels.pkl".format(ext_data_dir))


# Get basic statistic of tags, present as dataframe for better visualizing.
train_tags = get_stat(train_labels)
ori_df = pd.DataFrame(train_tags[0:],columns=['Tag','num'])
# ori_df


# Get basic statistic of new adding tags.
ext_tags = get_stat(ext)
ext_df = pd.DataFrame(ext_tags[0:],columns=['Tag','num'])
# Convert into dict format
ext_ = dict(ext_tags)


compare_df = get_compare_df(ori_df, train_tags, ext_data_dir, ext_, better_, worse_)
compare_df

,Tag,num,ext_data_1000_u_300,Better,Worse
0,GPE,2803,954,1,0
1,TIME,2585,157,1,0
2,NRP,1886,276,1,0
3,DUR,1634,0,0,1
4,LAN,1253,53,0,1
5,DATE,1098,52,1,0
6,PER,765,1056,1,0
7,FREQ,555,6,1,0
8,AGE,430,0,1,0
9,CARDINAL,412,282,1,0


### Check sentence and predictions

In [39]:
import os
from predict import Ner

base_clf = Ner(model_dir="baseline_model/")
ext_clf = Ner(model_dir="co-models/ext_2290_isw_model/")
# Load test data, which used for error analysis
test_sents = joblib.load("data/30-test-isw-sentences.pkl")
test_labels = joblib.load("data/30-test-isw-labels.pkl")

In [42]:
def select_pred_by_tag(selected_tag, test_sents, test_labels, base_clf, ext_clf):
    sele_ls = []
    for sent, true_tag in zip(test_sents[:20], test_labels[:20]):
        base_pred = base_clf.predict(sent)
        ext_pred = ext_clf.predict(sent)
        
        base_tag = [dic['tag'] for dic in base_pred]
        ext_tag = [dic['tag'] for dic in ext_pred]
        
        if any(x in base_tag for x in selected_tag) or any(x in ext_tag for x in selected_tag):
            print("sent", sent)
            print("True", true_tag)
            print("base", base_tag)
            print("ext ", ext_tag)
            print("")
            sele_ls.append((sent, true_tag, base_tag, ext_tag))
        else:
            pass
    return sele_ls

In [52]:
# Load test result of baseline and cotrain model
baseline_ls = load_eval_result("baseline_model/test_results.txt")

# _1521_ls = load_eval_result("co-models/ext_1000_isw_model/test_results.txt")
_2290_ls = load_eval_result("co-models/ext_2290_isw_model/test_results.txt")

# Shows the better and worse tags [[PER, 0.8, 0.9]]
better_, worse_ = get_better_worse_tags(baseline_ls, _2290_ls)

## The "selected_tag" can be the ones "better" or "worse"
better_tags = [tag for [tag, *_] in better_]
print("Better_tags:", better_tags)
print("")
worse_tags = [tag for [tag, *_] in worse_]
print("Worse_tags:", worse_tags)


Better_tags: ['AGE', 'ART', 'CARDINAL', 'DATE', 'EVT', 'FAC', 'FRAC', 'FREQ', 'GPE', 'NRP', 'PER', 'QUANT', 'RATE', 'SORD', 'TIME', 'macro avg']

Worse_tags: ['DUR', 'LAN', 'LOC', 'MISC', 'MON', 'ORDINAL', 'ORG', 'TITLE']


In [55]:
print("")
# tag can be either "better" or "worse" on ext model preds...
selected_tag = ["B-DATE", "I-DATE"]
sele_ls = select_pred_by_tag(selected_tag, test_sents, test_labels, base_clf, ext_clf)


sent Aber ich mein in größeren Massen kann man sagen dass der ökonomische Faktor erst im 20
True ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE']
base ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GPE']
ext  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE']

sent Und äh im 9
True ['O', 'O', 'O', 'B-DATE']
base ['O', 'O', 'O', 'B-GPE']
ext  ['O', 'O', 'O', 'B-DATE']



In [34]:
# Save into txt
analysis_ls = []
for sent, true_tag in zip(test_sents, test_labels):
    try:
        base_pred = base_clf.predict(sent)
        ext_pred = ext_clf.predict(sent)

        base_tag = [dic['tag'] for dic in base_pred]
        ext_tag = [dic['tag'] for dic in ext_pred]
        analysis_ls.append((sent, true_tag, base_tag, ext_tag))
    except:
        pass

with open("2290_analysis.txt", "w", encoding="utf-8") as writer:
    for (sent, true_tag, base_tag, ext_tag) in analysis_ls:
        writer.write("sent    "+str(sent)+'\n')
        writer.write("True Tag"+str(true_tag)+'\n')
        writer.write("baseline"+str(base_tag)+'\n')
        writer.write("ext_tag "+str(ext_tag)+'\n')
        writer.write('\n')
writer.close()

Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
